## Modelling for BME - Inverse problem
### Practice 3
### Student: Eduardo Osquel Perez Rivero

First, we prepare all the necessary libraries

In [ ]:
# Install necessary packages
!pip install numpy pmcx jdata bjdata matplotlib

Then, we import them into our notebook to start working with them

In [ ]:
# Import libraries
import numpy as np
import pmcx
import jdata as jd
import matplotlib.pyplot as plt
import h5py
from scipy.sparse import lil_matrix, linalg, diags
from scipy.linalg import pinv

The next step is to check that we have a GPU to be able to use the tools offered by MonteCarlo

In [ ]:
# Print pmcx version
print("pmcx version:", pmcx.__version__)

# Check GPU availability
pmcx.gpuinfo()

We first read the data from MATLAB provided dataframes

In [ ]:
# Load data from 'A.mat'
with h5py.File('A.mat', 'r') as f:
    arrays_A = {k: np.array(v) for k, v in f.items()}
A = arrays_A['A']

# Load data from 'y_mes.mat'
with h5py.File("y_mes.mat", 'r') as f:
    arrays_y_mes = {k: np.array(v) for k, v in f.items()}
y_mes = arrays_y_mes['y_mes']

Next, we check the shape and the condition number of matrix A

In [ ]:
print(f"The shape of A matrix is {A.shape}")
print(f"The condition number of A matrix is {np.linalg.cond(A)}")

From the results above, we can see that the shape of matrix A is 2D, with 15625 rows and 5625 columns. The condition number of A is around 1009987, indicating a very large value. This large condition number suggests that A is closer to being a singular matrix, meaning it is highly ill-conditioned. A high condition number implies that small changes in the input may lead to significant changes in the output. Conversely, a lower condition number indicates that the matrix is relatively stable in response to input changes.

The next step involves defining vector x to obtain y = Ax. The transpose of y is 5625 * 1, and the transpose of A is 5625 * 15625. Therefore, we define x as a vector with a shape of 15625 * 1. To compare the result of the inverse with the true value of x, we attempt to define the true value of x as a vector where each element is equal to 2.

In [ ]:
x = np.ones((15625*1)) + 1
x.shape

Then we can get y and y_noise

In [ ]:
y = np.dot(np.transpose(A) ,x)
y = y.reshape((5625,1))

n = np.random.randn(5625,1)
n.shape

y_noise = y + n
y_noise

Because matrix A is irreversible and singular, we cannot use $$x=A ^{-1} * y$$ to obtain x.<br />

In [ ]:
A.shape

Hence, we'll employ an alternative method to solve the inverse problem and then compare the outcome with the true value of x. For this purpose, we will solve the inverse problem both with and without the inclusion of noise, which will allow us to observe the MSE anomalies.

Without noise:

In [ ]:
x_without_noise = np.linalg.lstsq(A.transpose() , y , rcond=None)[0]
x_without_noise

mse = np.mean((x - x_without_noise) ** 2)
mse

With noise:

In [ ]:
x_with_noise = np.linalg.lstsq(A.transpose() , y_noise , rcond=None)[0]
x_with_noise

mse = np.mean((x - x_with_noise) ** 2)
mse

When we directly apply the middle least squares method without addressing the noise, we may encounter MSE anomalies. Hence, it's imperative to account for the noise's influence on the results when tackling inverse problems, especially in the presence of noise. Regularization serves as a crucial tool to mitigate the impact of noise, thus enhancing the accuracy of our solutions to the inverse problem

In this case, we employ Tikhonov Regularization (L2) to accomplish this task. Also, after computing the regularization term, we utilize the function pinv to compute the pseudo-inverse of the matrix tikhonov_one

In [ ]:
alpha = 0.1  # Set regularization parameter
regularization_matrix = alpha * np.eye(A.shape[1])  # Create regularization matrix
tikhonov_one = pinv(np.dot(A.T, A) + regularization_matrix)  # Calculate Tikhonov regularization matrix

tikhonov_two = np.dot(tikhonov_one, A.T)

tikhonov_two.shape = tikhonov_two.shape
print("Shape of tikhonov_two:", tikhonov_two.shape)

Subsequently, we obtain tikhonov_two by performing the matrix multiplication tikhonov_one * A.T

In [ ]:
x_reg = np.dot(tikhonov_two.T, y)

mse = np.mean((x - x_reg) ** 2)
mse

Upon incorporating regularization, we observe that the mean squared error (MSE) regression remains consistent, approximately at 72.36%. When contrasted with the scenario devoid of noise, both the computational error and MSE exhibit relatively larger magnitudes owing to the noise influence. In comparison to cases where regularization is not employed for processing, the utilization of ridge regression demonstrates normal MSE regression behavior, indicating enhanced accuracy in solving the inverse problem.

## Exercise 2

We will use "A" and "y_mes" to solve inverse problem:$$y = Ax$$
First we use SVD + L1 regularization to solve inverse problem
  $$A = USV^{T}$$
  so we can get $$x = VS^{-1}U^ty$$

In [ ]:
U, S, V = np.linalg.svd(A, full_matrices=0)  # Perform SVD decomposition
alpha = 0.1  # L1 regularization parameter
S_inv = np.diag(1 / (S + alpha))  # Calculate inverse singular values
tikhonov_one = np.dot(V.T, S_inv)  # Compute tikhonov_one
tikhonov_one.shape

tikhonov_two = np.dot(tikhonov_one, U.T)
tikhonov_two.shape

x1 = np.dot(tikhonov_two.T, y_mes.T)
x1.shape

Then we add positive constrains to the A matrix

In [ ]:
pre_A = A
for i in range(15625):
    for j in range(5625):
        if(A[i][j]) < 0:
            A[i][j] = 0

Subsequently, we employ Singular Value Decomposition (SVD) along with L1 regularization to revisit the inverse problem

In [ ]:
U, S, V = np.linalg.svd(pre_A, full_matrices=0)  # Perform SVD decomposition
alpha = 0.1  # L1 regularization parameter
S_inv = np.diag(1 / (S + alpha))  # Calculate inverse singular values
tikhonov_one = np.dot(V.T, S_inv)  # Compute tikhonov_one
tikhonov_one.shape  # Get the shape of tikhonov_one

tikhonov_two = np.dot(tikhonov_one, U.T)
tikhonov_two.shape

x2 = np.dot(tikhonov_two.T, y_mes.T)
x2.shape

Now we can visualize the degree of approximation between Ax and y_mes by plotting a graph

In [ ]:
dt_true = np.dot(A.T, x1).flatten()
dt_observed = y_mes.T.flatten()

# Plotting the data
plt.plot(dt_true, color='blue', marker="o", label="dt_true")
plt.plot(dt_observed, color='red', marker="o", label="dt_observed")

# Adding labels and legend
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()

# Display the plot
plt.show()

The similarity between the calculated x and the measured y_mes in terms of distribution is evident. Subsequently, we proceed to calculate the residuals and conduct further analysis. Upon merging the two figures above, it is apparent that the images almost perfectly coincide. This indicates a strong agreement between the calculated and measured data distributions

In [ ]:
residuals = dt_observed - dt_true

plt.figure(figsize=(8, 6))
plt.scatter(range(len(residuals)), residuals, color='b', label='Residuals')
plt.axhline(y=0, color='r', linestyle='--', linewidth=1)  # Add a horizontal line representing zero residuals
plt.xlabel('Data Point')
plt.ylabel('Residual')
plt.title('Residual Plot')
plt.legend()
plt.grid(True)
plt.show()

The residual plot size, at 1e-7, signifies a close match between the measured result y_mes and the calculated x, indicating high precision in the results obtained through SVD decomposition and L1 regularization. Before tackling the inverse problem again, we delve into the ill-posed nature of matrix A. If A proves to be invertible, we can directly multiply the inverse of the A matrix on both sides of the equation to obtain the inverse problem solution. However, if A is singular, alternative methods must be employed for solving it. Analyzing the condition number of matrix A provides insights into its pathological nature. A larger condition number indicates a more severe pathological state for the matrix, where even minor perturbations in measurement can significantly affect the final inverse problem solution. Moreover, in the presence of noise, the ill-posedness of matrix A is further exacerbated, leading to reduced accuracy and stability of the results